In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
# tf.enable_eager_execution() #1.X版本下调用
from keras import backend as K
K.set_image_data_format('channels_first')
import os
os.chdir('./4-4/face_recognition/')
import fr_utils
from inception_blocks_v2 import *

Using TensorFlow backend.


In [2]:
FRmodel = faceRecoModel((3,96,96))

In [3]:
FRmodel.count_params()

3743280

In [4]:
FRmodel.summary()

____________________
activation_14 (Activation)      (None, 64, 12, 12)   0           inception_3b_pool_bn[0][0]       
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 64, 12, 12)   0           inception_3b_1x1_bn[0][0]        
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 320, 12, 12)  0           activation_11[0][0]              
                                                                 activation_13[0][0]              
                                                                 activation_14[0][0]              
                                                                 activation_15[0][0]              
__________________________________________________________________________________________________
inception_3c_3x3_conv1 (Conv2D) (None, 128, 12, 12)  41088       concatenate_2[0][0]    

In [5]:
def triplet_loss(y_true, y_pred, alpha=0.2):

    '''
    input:
    y_true - ...keras.losses.[loss_function](y_true, y_pred)
    y_pred - [encode(anchor), encode(positive), encode(negetive)], ~(None,128)
    alpha - margin
    
    output:
    loss - float
    '''

    anchor, positive, negetive = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negetive), axis=-1)
    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))

    return loss

In [6]:
FRmodel.compile(optimizer='adam', loss=triplet_loss, metrics=['accuracy'])
fr_utils.load_weights_from_FaceNet(FRmodel)

In [7]:
database = {}
database["danielle"] = fr_utils.img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = fr_utils.img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = fr_utils.img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = fr_utils.img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = fr_utils.img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = fr_utils.img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = fr_utils.img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = fr_utils.img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = fr_utils.img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = fr_utils.img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = fr_utils.img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = fr_utils.img_to_encoding("images/arnaud.jpg", FRmodel)

In [43]:
def verify(image_path, identity, database, model):
    
    encoding = fr_utils.img_to_encoding(image_path, model)
    dist = np.linalg.norm(encoding - database[identity])
    
    if dist < 0.325:
        print(f'欢迎{identity}回家!')
        is_door_open = True
    else:
        print(f'经验证，您与{identity}不符!')
        is_door_open = False
    
    return dist, is_door_open

In [44]:
verify("images/camera_0.jpg","younes",database,FRmodel)

欢迎younes回家!


(0.3242399, True)

In [45]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

经验证，您与kian不符!


(0.4915761, False)

In [46]:
verify("images/camera_1.jpg", "bertrand", database, FRmodel)

欢迎bertrand回家!


(0.22910224, True)

In [47]:
verify("images/camera_1.jpg", "arnaud", database, FRmodel)

欢迎arnaud回家!


(0.24459074, True)